# Voice Chat Server với Bank Model

Notebook này tạo một server realtime để chat bằng giọng nói với model bank-model-2b.
Tính năng:
- 🎙️ Nhận giọng nói (Speech-to-Text) - chỉ voice, không có ảnh
- 🤖 Xử lý với model Qwen3-VL-2B
- 🔊 Trả lời bằng giọng nói (Text-to-Speech) realtime
- 📊 Hiển thị tài nguyên hệ thống (CPU, RAM, GPU) realtime


In [1]:
# Kiểm tra và upgrade transformers nếu cần (cho Qwen3VL)
import subprocess
import sys

# Kiểm tra version transformers hiện tại
try:
    import transformers
    print(f"📦 Transformers version hiện tại: {transformers.__version__}")
    
    # Kiểm tra xem có Qwen3VL không
    try:
        from transformers import Qwen3VLForConditionalGeneration
        print("✅ Qwen3VLForConditionalGeneration đã có sẵn trong transformers")
    except ImportError:
        print("❌ Qwen3VLForConditionalGeneration chưa có trong transformers")
        print("💡 Đang upgrade transformers lên bản mới nhất...")
        # Thử cài bản mới nhất trước
        result = subprocess.run([sys.executable, "-m", "pip", "install", "-q", "transformers", "--upgrade", "--force-reinstall"], check=False)
        if result.returncode == 0:
            print("✅ Đã upgrade transformers lên bản mới nhất")
            print("💡 Vui lòng restart runtime và chạy lại cell này")
        else:
            print("⚠️  Không thể cài bản mới nhất, thử version tương thích với coqui-tts...")
            subprocess.run([sys.executable, "-m", "pip", "install", "-q", "transformers>=4.52.1,<4.56", "--upgrade"], check=False)
            print("✅ Đã upgrade transformers (version tương thích với coqui-tts)")
            print("💡 Lưu ý: Version transformers được giới hạn <4.56 để tương thích với coqui-tts")
except ImportError:
    print("⚠️  Transformers chưa được cài, đang cài đặt bản mới nhất...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "transformers", "--upgrade", "--force-reinstall"], check=False)
    print("✅ Đã cài transformers (bản mới nhất)")


## 1. Cài đặt Dependencies


In [2]:
# Cài đặt các thư viện cần thiết
# Theo tài liệu Qwen: https://qwen.readthedocs.io/en/latest/
# Cài đặt transformers bản mới nhất
# Lưu ý: Bản mới nhất có thể không tương thích với coqui-tts
# Nếu gặp lỗi với coqui-tts, có thể cần downgrade về >=4.52.1,<4.56
print("Đang cài đặt transformers bản mới nhất...")
%pip install -q transformers --upgrade --force-reinstall
print("✅ Transformers đã được cài đặt (bản mới nhất)")
print("💡 Nếu gặp lỗi với coqui-tts, có thể cần: %pip install transformers>=4.52.1,<4.56")

# Cài torch với version tương thích với coqui-tts
# coqui-tts yêu cầu torch<2.9,>=2.1
print("Đang cài đặt torch (tương thích với coqui-tts/XTTS)...")
%pip install -q "torch>=2.1,<2.9" "torchaudio>=2.1.0,<2.9"
print("✅ Torch đã được cài đặt (version tương thích với coqui-tts)")
%pip install -q gradio
%pip install -q openai-whisper
%pip install -q pydub
%pip install -q soundfile
%pip install -q accelerate
%pip install -q bitsandbytes  # Cho quantization (theo Qwen docs)
%pip install -q psutil  # Cho monitoring tài nguyên

# Flash Attention 2 (tùy chọn, để tăng tốc - theo Qwen docs)
# Uncomment dòng dưới nếu muốn cài Flash Attention 2 (cần compile, có thể mất thời gian)
# %pip install -q flash-attn --no-build-isolation
print("💡 Tip: Có thể cài flash-attn để tăng tốc (tùy chọn): pip install flash-attn --no-build-isolation")

# Cài TTS (Coqui TTS) - thử nhiều cách
print("Đang cài TTS (Coqui TTS)...")
import subprocess
import sys

# Option 1: Thử cài TTS package trực tiếp từ PyPI
print("Thử cài TTS từ PyPI...")
result = subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q", "TTS"], 
    capture_output=True, 
    text=True,
    timeout=300
)

if result.returncode != 0:
    print("⚠️  Không thể cài TTS từ PyPI, thử cài từ source...")
    # Option 2: Cài từ source (GitHub)
    result2 = subprocess.run(
        [sys.executable, "-m", "pip", "install", "-q", 
         "git+https://github.com/coqui-ai/TTS.git"], 
        capture_output=True,
        text=True,
        timeout=600
    )
    if result2.returncode != 0:
        print("⚠️  Không thể cài TTS từ source")

# Kiểm tra xem TTS đã được cài chưa
try:
    from TTS.api import TTS
    print("✅ TTS đã được cài thành công!")
except ImportError as e:
    print(f"⚠️  TTS không khả dụng: {e}")
    print("Sẽ thử cài lại khi load model...")
    
    # Fallback: cài pyttsx3 nếu TTS không được
    print("Đang cài pyttsx3 làm fallback...")
    try:
        subprocess.run(
            [sys.executable, "-m", "pip", "install", "-q", "pyttsx3"], 
            capture_output=True,
            timeout=60
        )
        print("✅ pyttsx3 đã được cài (có thể cần eSpeak)")
    except:
        print("⚠️  Không thể cài pyttsx3")


## 2. Import Libraries


In [3]:
import torch
import whisper

# Import transformers với error handling
try:
    from transformers import Qwen3VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig, TextIteratorStreamer
    print("✅ Đã import Qwen3VLForConditionalGeneration thành công")
except ImportError as e:
    print(f"❌ Lỗi import Qwen3VLForConditionalGeneration: {e}")
    print("💡 Vui lòng:")
    print("   1. Chạy cell trước đó để upgrade transformers")
    print("   2. Restart runtime (Runtime > Restart runtime)")
    print("   3. Chạy lại cell này")
    print("💡 Hoặc chạy lệnh: !pip install transformers --upgrade --force-reinstall")
    raise

import gradio as gr
import io
import base64
import os
from typing import Optional, Tuple
import numpy as np
import tempfile
from functools import lru_cache
import psutil  # Cho monitoring tài nguyên
import time
import re  # Cho text processing
from threading import Thread  # Cho streaming

# Import TTS với fallback
TTS_AVAILABLE = False
pyttsx3_available = False

try:
    from TTS.api import TTS
    TTS_AVAILABLE = True
    print("Coqui TTS đã sẵn sàng!")
except ImportError:
    print("Coqui TTS không khả dụng, sẽ thử pyttsx3...")
    try:
        import pyttsx3
        pyttsx3_available = True
        print("pyttsx3 đã sẵn sàng!")
    except:
        print("Cả TTS và pyttsx3 đều không khả dụng. Cần cài một trong hai.")


## 3. Hàm Monitoring Tài nguyên


In [4]:
def get_system_resources():
    """
    Lấy thông tin tài nguyên hệ thống (CPU, RAM, GPU)
    """
    # CPU
    cpu_percent = psutil.cpu_percent(interval=0.1)
    cpu_count = psutil.cpu_count()
    
    # RAM
    memory = psutil.virtual_memory()
    ram_total_gb = memory.total / (1024**3)
    ram_used_gb = memory.used / (1024**3)
    ram_available_gb = memory.available / (1024**3)
    ram_percent = memory.percent
    
    # GPU (nếu có)
    gpu_info = ""
    gpu_memory_used = 0
    gpu_memory_total = 0
    gpu_memory_percent = 0
    
    if torch.cuda.is_available():
        gpu_count = torch.cuda.device_count()
        gpu_info = f"GPU: {torch.cuda.get_device_name(0)}"
        gpu_memory_used = torch.cuda.memory_allocated(0) / (1024**3)
        gpu_memory_total = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        gpu_memory_percent = (gpu_memory_used / gpu_memory_total) * 100
    else:
        gpu_info = "GPU: Không có"
    
    return {
        "cpu_percent": cpu_percent,
        "cpu_count": cpu_count,
        "ram_total_gb": ram_total_gb,
        "ram_used_gb": ram_used_gb,
        "ram_available_gb": ram_available_gb,
        "ram_percent": ram_percent,
        "gpu_info": gpu_info,
        "gpu_memory_used": gpu_memory_used,
        "gpu_memory_total": gpu_memory_total,
        "gpu_memory_percent": gpu_memory_percent,
    }

def format_resources_info():
    """
    Format thông tin tài nguyên thành string để hiển thị
    """
    res = get_system_resources()
    
    info = f"""
### 📊 Tài nguyên hệ thống:

**CPU:**
- Sử dụng: {res['cpu_percent']:.1f}% / {res['cpu_count']} cores
- Còn lại: {100 - res['cpu_percent']:.1f}%

**RAM:**
- Tổng: {res['ram_total_gb']:.2f} GB
- Đã dùng: {res['ram_used_gb']:.2f} GB ({res['ram_percent']:.1f}%)
- Còn lại: {res['ram_available_gb']:.2f} GB ({100 - res['ram_percent']:.1f}%)

**{res['gpu_info']}**
"""
    
    if torch.cuda.is_available():
        info += f"""
- Tổng: {res['gpu_memory_total']:.2f} GB
- Đã dùng: {res['gpu_memory_used']:.2f} GB ({res['gpu_memory_percent']:.1f}%)
- Còn lại: {res['gpu_memory_total'] - res['gpu_memory_used']:.2f} GB ({100 - res['gpu_memory_percent']:.1f}%)
"""
    
    return info


## 4. Authentication với Hugging Face (nếu cần)

Nếu model là private hoặc bạn muốn tải nhanh hơn, có thể đăng nhập Hugging Face.


In [ ]:
# Uncomment và thêm token của bạn nếu cần (cho model private hoặc tải nhanh hơn)
# from huggingface_hub import login
# login(token="YOUR_HF_TOKEN_HERE")


## 4. Load Model và Speech Recognition


## 5. Cấu hình Full Công Suất cho Model (Tùy chọn)

Nếu GPU đủ lớn (>=20GB), có thể tắt quantization và tối ưu để chạy full công suất.


In [6]:
# Kiểm tra và tối ưu model cho full công suất
# Chỉ chạy sau khi model đã được load
if 'model' in globals() and model is not None:
    if torch.cuda.is_available():
        gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        gpu_name = torch.cuda.get_device_name(0)
        print(f"GPU: {gpu_name}")
        print(f"GPU Memory: {gpu_memory_gb:.2f} GB")
        
        # Kiểm tra xem model có đang dùng quantization không
        is_quantized = hasattr(model, 'hf_quantizer') or hasattr(model, 'quantization_config')
        
        # Nếu GPU >= 20GB và model đang dùng quantization, có thể reload không quantization
        if gpu_memory_gb >= 20 and is_quantized:
            print("\n⚠️  Model đang dùng quantization nhưng GPU đủ lớn.")
            print("   Có thể reload model không quantization để full công suất (tùy chọn)")
            print("   Uncomment code bên dưới để reload:")
            print("""
# Uncomment để reload model full precision:
# model = Qwen3VLForConditionalGeneration.from_pretrained(
#     model_name,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=True,
#     attn_implementation="flash_attention_2",
# )
            """)
        elif gpu_memory_gb >= 20:
            print("✅ Model đã được load với full precision - sẵn sàng full công suất!")
        else:
            print("⚠️  GPU nhỏ, model đang dùng quantization (phù hợp)")
    else:
        print("⚠️  Không có GPU, model đang dùng quantization (phù hợp)")
else:
    print("⚠️  Model chưa được load. Chạy cell load model trước.")


In [7]:
# Load Whisper model cho Speech-to-Text (dùng tiny để nhanh hơn)
# Tối ưu: Force load lên GPU nếu có
print("Đang load Whisper model (tiny - nhanh nhất)...")
if torch.cuda.is_available():
    print(f"🚀 GPU có sẵn: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")
    whisper_model = whisper.load_model("tiny", device="cuda")  # Force lên GPU
    print("✅ Whisper model đã load trên GPU!")
else:
    whisper_model = whisper.load_model("tiny", device="cpu")
    print("⚠️  Whisper model đã load trên CPU (không có GPU)")

# Load Bank Model từ Hugging Face
print("\n" + "="*50)
print("Đang tải Bank Model từ Hugging Face...")
print("="*50)

model_name = "hainguyen306201/bank-model-2b"
print(f"Model: {model_name}")

# Kiểm tra xem model có tồn tại không
try:
    from huggingface_hub import model_info
    info = model_info(model_name)
    print(f"✅ Model tìm thấy trên Hugging Face!")
    print(f"   - Model ID: {info.modelId}")
    print(f"   - Files: {len(info.siblings)} files")
except Exception as e:
    print(f"⚠️  Không thể kiểm tra model info: {e}")
    print("   Tiếp tục tải model...")

# Cấu hình quantization 4-bit để tiết kiệm memory và tăng tốc
# Theo tài liệu Qwen: https://qwen.readthedocs.io/en/latest/
print("\nĐang cấu hình quantization (4-bit)...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # NF4 quantization - chuẩn Qwen
    bnb_4bit_compute_dtype=torch.bfloat16,  # bfloat16 cho tính toán
    bnb_4bit_use_double_quant=True,  # Double quantization để tiết kiệm hơn
)

# Load model với các tùy chọn để đảm bảo tải đúng và chạy trên GPU
print("\nĐang tải model (có thể mất vài phút lần đầu)...")

# Kiểm tra GPU và quyết định quantization
if torch.cuda.is_available():
    gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {gpu_memory_gb:.2f} GB")
    
    # Nếu GPU >= 20GB, có thể load không quantization để nhanh hơn
    if gpu_memory_gb >= 20:
        print("✅ GPU đủ lớn, sẽ load model không quantization (full precision) để tối ưu tốc độ!")
        use_quantization = False
    else:
        print("⚠️  GPU nhỏ, sẽ dùng quantization 4-bit để tiết kiệm memory")
        use_quantization = True
else:
    print("⚠️  Không có GPU, sẽ dùng quantization 4-bit")
    use_quantization = True

try:
    if use_quantization:
        # Dùng quantization cho GPU nhỏ hoặc CPU
        model = Qwen3VLForConditionalGeneration.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",  # Tự động đặt lên GPU nếu có
            torch_dtype=torch.bfloat16,
            trust_remote_code=True,
            resume_download=True,
            force_download=False,
        )
    else:
        # Load full precision trên GPU lớn (nhanh hơn)
        # Theo tài liệu Qwen: sử dụng attn_implementation="flash_attention_2" nếu hỗ trợ
        # Kiểm tra xem có flash-attn không
        try:
            import flash_attn
            use_flash_attention = True
            print("✅ Flash Attention 2 được phát hiện, sẽ sử dụng để tối ưu tốc độ")
        except ImportError:
            use_flash_attention = False
            print("⚠️  Flash Attention 2 chưa được cài, sử dụng attention mặc định")
            print("   Có thể cài: pip install flash-attn (tùy chọn, để tăng tốc)")
        
        model = Qwen3VLForConditionalGeneration.from_pretrained(
            model_name,
            device_map="auto",  # Tự động đặt lên GPU
            torch_dtype=torch.bfloat16,  # bfloat16 cho hiệu suất tốt
            trust_remote_code=True,  # Cần thiết cho Qwen models
            resume_download=True,
            force_download=False,
            attn_implementation="flash_attention_2" if use_flash_attention else "sdpa",  # Flash Attention 2 hoặc SDPA
        )
    
    # Đảm bảo model trên GPU và tối ưu
    if torch.cuda.is_available():
        model_device = next(model.parameters()).device
        print(f"✅ Model đã được tải và load thành công!")
        print(f"   Model device: {model_device}")
        if model_device.type == "cuda":
            print(f"   ✅ Model đang chạy trên GPU: {torch.cuda.get_device_name(model_device.index)}")
            # Tối ưu: Compile model nếu PyTorch >= 2.0 (tăng tốc ~20-30%)
            try:
                if hasattr(torch, 'compile') and torch.__version__ >= "2.0.0":
                    print("   🔧 Đang compile model để tăng tốc (PyTorch 2.0+)...")
                    model = torch.compile(model, mode="reduce-overhead")
                    print("   ✅ Model đã được compile!")
            except Exception as e:
                print(f"   ⚠️  Không thể compile model: {e} (không ảnh hưởng chức năng)")
        else:
            print(f"   ⚠️  Model đang chạy trên {model_device.type}, đang chuyển lên GPU...")
            model = model.to("cuda")
            # Thử compile sau khi move
            try:
                if hasattr(torch, 'compile') and torch.__version__ >= "2.0.0":
                    model = torch.compile(model, mode="reduce-overhead")
                    print("   ✅ Model đã được compile!")
            except:
                pass
    else:
        print("✅ Model đã được tải và load thành công trên CPU!")
        
except Exception as e:
    print(f"❌ Lỗi khi tải model: {e}")
    print("\nThử tải lại với force_download=True...")
    if use_quantization:
        model = Qwen3VLForConditionalGeneration.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            trust_remote_code=True,
            force_download=True,
        )
    else:
        # Retry với flash attention nếu có
        try:
            import flash_attn
            use_flash_attention = True
        except ImportError:
            use_flash_attention = False
        
        model = Qwen3VLForConditionalGeneration.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            trust_remote_code=True,
            force_download=True,
            attn_implementation="flash_attention_2" if use_flash_attention else "sdpa",
        )
    print("✅ Model đã được tải lại thành công!")
    
    # Đảm bảo model trên GPU
    if torch.cuda.is_available():
        model = model.to("cuda")
        print(f"✅ Model đã được chuyển lên GPU!")

# Load processor - theo chuẩn Qwen
print("\nĐang tải processor...")
processor = AutoProcessor.from_pretrained(
    model_name, 
    trust_remote_code=True,  # Cần thiết cho Qwen models
    resume_download=True,
)
print("✅ Processor đã load!")

# Kiểm tra processor có đúng không
if not hasattr(processor, 'apply_chat_template'):
    print("⚠️  Processor không có apply_chat_template, có thể có vấn đề")
else:
    print("✅ Processor có apply_chat_template - OK")

# Kiểm tra tokenizer
if hasattr(processor, 'tokenizer'):
    print(f"✅ Tokenizer: {type(processor.tokenizer).__name__}")
    if hasattr(processor.tokenizer, 'eos_token_id') and processor.tokenizer.eos_token_id:
        print(f"   EOS token ID: {processor.tokenizer.eos_token_id}")
    else:
        print("   ⚠️  EOS token ID không được set, sẽ dùng pad_token_id")

# Kiểm tra và hiển thị thông tin GPU
print("\n" + "="*50)
print("📊 Thông tin GPU và Model:")
print("="*50)
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")
    
    # Kiểm tra model đã được load chưa
    try:
        if 'model' in globals() and model is not None:
            model_device = next(model.parameters()).device
            if model_device.type == "cuda":
                print(f"   ✅ Model đang chạy trên GPU: {torch.cuda.get_device_name(model_device.index)}")
                print(f"   GPU Memory đã dùng: {torch.cuda.memory_allocated(0) / (1024**3):.2f} GB")
            else:
                print(f"   ⚠️  Model đang chạy trên {model_device.type}")
        else:
            print(f"   ⚠️  Model chưa được load")
    except Exception as e:
        print(f"   ⚠️  Không thể kiểm tra model device: {e}")
    
    # Kiểm tra Whisper model
    try:
        if 'whisper_model' in globals() and whisper_model is not None:
            if hasattr(whisper_model, 'encoder') and hasattr(whisper_model.encoder, 'parameters'):
                whisper_device = next(whisper_model.encoder.parameters()).device
                if whisper_device.type == "cuda":
                    print(f"   ✅ Whisper đang chạy trên GPU")
                else:
                    print(f"   ⚠️  Whisper đang chạy trên CPU")
            else:
                # Nếu không kiểm tra được, giả định đã load đúng device
                print(f"   ✅ Whisper đã được load với device phù hợp")
        else:
            print(f"   ⚠️  Whisper model chưa được load")
    except Exception as e:
        print(f"   ℹ️  Không thể kiểm tra Whisper device: {e}")
else:
    print("⚠️  Không có GPU, tất cả đang chạy trên CPU")

print("\n" + "="*50)
print("✅ Bank Model đã sẵn sàng để sử dụng!")
print("="*50)

# Tối ưu: Warmup model để tăng tốc lần đầu generate
print("\n🔥 Đang warmup model (lần đầu có thể chậm)...")
try:
    # Warmup với một câu ngắn
    warmup_messages = [{"role": "user", "content": [{"type": "text", "text": "Hello"}]}]
    warmup_inputs = processor.apply_chat_template(
        warmup_messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    )
    if torch.cuda.is_available():
        warmup_inputs = {k: v.to("cuda") if isinstance(v, torch.Tensor) else v for k, v in warmup_inputs.items()}
    
    model.eval()
    with torch.inference_mode():
        _ = model.generate(
            **warmup_inputs,
            max_new_tokens=10,
            do_sample=False,
            use_cache=True,
        )
    print("✅ Model đã được warmup - sẵn sàng generate nhanh!")
except Exception as e:
    print(f"⚠️  Warmup không thành công (không ảnh hưởng chức năng): {e}")

# Load TTS offline (Coqui TTS hoặc pyttsx3) - Force GPU nếu có
print("\n" + "="*50)
print("Đang load TTS model (offline)...")
print("="*50)
tts = None
tts_type = None

# Đảm bảo TTS chạy trên GPU nếu có
use_gpu_tts = torch.cuda.is_available()
if use_gpu_tts:
    print(f"🚀 TTS sẽ chạy trên GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️  TTS sẽ chạy trên CPU (không có GPU)")

# Kiểm tra lại TTS_AVAILABLE và thử cài lại nếu cần
if not TTS_AVAILABLE:
    print("⚠️  Coqui TTS chưa được cài đặt, đang thử cài lại...")
    import subprocess
    import sys
    try:
        # Thử cài TTS từ PyPI
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", "-q", "TTS"],
            capture_output=True,
            text=True,
            timeout=300
        )
        if result.returncode == 0:
            print("✅ Đã cài TTS từ PyPI, đang import lại...")
            try:
                from TTS.api import TTS
                TTS_AVAILABLE = True
                print("✅ Coqui TTS đã sẵn sàng!")
            except ImportError:
                print("⚠️  Vẫn không thể import TTS sau khi cài")
        else:
            print("⚠️  Không thể cài TTS từ PyPI, thử cài từ source...")
            result2 = subprocess.run(
                [sys.executable, "-m", "pip", "install", "-q", "git+https://github.com/coqui-ai/TTS.git"],
                capture_output=True,
                text=True,
                timeout=600
            )
            if result2.returncode == 0:
                print("✅ Đã cài TTS từ source, đang import lại...")
                try:
                    from TTS.api import TTS
                    TTS_AVAILABLE = True
                    print("✅ Coqui TTS đã sẵn sàng!")
                except ImportError:
                    print("⚠️  Vẫn không thể import TTS sau khi cài từ source")
    except Exception as e:
        print(f"⚠️  Lỗi khi cài TTS: {e}")

if TTS_AVAILABLE:
    try:
        # Thử load XTTS v2 (chất lượng cao, hỗ trợ nhiều ngôn ngữ)
        tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2", gpu=use_gpu_tts)
        tts_type = "coqui_xtts"
        if use_gpu_tts:
            print("✅ TTS model (XTTS v2) đã load trên GPU!")
        else:
            print("✅ TTS model (XTTS v2) đã load trên CPU!")
    except Exception as e:
        print(f"Không thể load XTTS v2: {e}")
        # Fallback: dùng model tiếng Việt nhẹ hơn
        print("Đang load TTS model tiếng Việt (nhẹ hơn)...")
        try:
            tts = TTS(model_name="tts_models/vi/vietnamese", gpu=use_gpu_tts)
            tts_type = "coqui_vi"
            if use_gpu_tts:
                print("✅ TTS model tiếng Việt đã load trên GPU!")
            else:
                print("✅ TTS model tiếng Việt đã load trên CPU!")
        except Exception as e2:
            print(f"Không thể load TTS tiếng Việt: {e2}")
            # Fallback cuối: dùng model đơn giản nhất
            print("Đang load TTS model đơn giản...")
            try:
                tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", gpu=use_gpu_tts)
                tts_type = "coqui_en"
                if use_gpu_tts:
                    print("✅ TTS model đơn giản đã load trên GPU!")
                else:
                    print("✅ TTS model đơn giản đã load trên CPU!")
            except Exception as e3:
                print(f"Không thể load Coqui TTS: {e3}")
                tts = None

# Fallback: dùng pyttsx3 nếu TTS không khả dụng
if tts is None and pyttsx3_available:
    try:
        tts = pyttsx3.init()
        tts_type = "pyttsx3"
        # Cấu hình giọng nói
        voices = tts.getProperty('voices')
        # Tìm giọng tiếng Việt nếu có
        for voice in voices:
            if 'vietnamese' in voice.name.lower() or 'vi' in voice.id.lower():
                tts.setProperty('voice', voice.id)
                break
        print("TTS model (pyttsx3) đã load!")
    except Exception as e:
        print(f"Không thể load pyttsx3: {e}")
        tts = None

if tts is None:
    print("\n" + "="*50)
    print("⚠️  CẢNH BÁO: Không có TTS nào khả dụng!")
    print("="*50)
    print("Có thể thử các cách sau:")
    print("1. Chạy lại cell này để thử cài TTS tự động")
    print("2. Cài thủ công: %pip install TTS")
    print("3. Hoặc cài từ source: %pip install git+https://github.com/coqui-ai/TTS.git")
    print("4. Hoặc cài pyttsx3: %pip install pyttsx3")
    print("\nLưu ý: Server vẫn hoạt động nhưng sẽ không có TTS (không phát âm)")
    print("="*50)


## 4. Hàm xử lý Speech-to-Text


In [8]:
def speech_to_text(audio_path: Optional[str]) -> str:
    """
    Chuyển đổi file audio thành text sử dụng Whisper (tối ưu tốc độ)
    Tối ưu: Greedy decoding, không beam search, tối ưu I/O, chạy trên GPU
    """
    # Kiểm tra whisper_model đã được load chưa
    if 'whisper_model' not in globals() or whisper_model is None:
        return "[Lỗi: Whisper model chưa được load]"
    
    if audio_path is None:
        return ""
    
    try:
        # Đảm bảo Whisper model trên GPU nếu có
        # Whisper model có thể không có .parameters(), kiểm tra device khác cách
        if torch.cuda.is_available():
            # Whisper tự động sử dụng GPU nếu load với device="cuda"
            # Không cần move thủ công vì đã load với device="cuda" ở trên
            pass
        
        # Tối ưu: Sử dụng greedy decoding (beam_size=1) để nhanh nhất
        # Nếu cần chất lượng hơn, có thể tăng beam_size
        use_fp16 = torch.cuda.is_available()  # Sử dụng fp16 nếu có GPU
        result = whisper_model.transcribe(
            audio_path,
            language="vi",
            fp16=use_fp16,  # Sử dụng fp16 nếu có GPU
            verbose=False,  # Tắt verbose để nhanh hơn
            condition_on_previous_text=False,  # Tắt để nhanh hơn
            initial_prompt="Đây là một cuộc trò chuyện bằng tiếng Việt.",  # Prompt ngắn
            compression_ratio_threshold=2.4,
            logprob_threshold=-1.0,
            no_speech_threshold=0.6,
            beam_size=1,  # Greedy decoding - nhanh nhất
            best_of=1,  # Không cần best_of khi beam_size=1
            temperature=0.0,  # Deterministic - nhanh hơn
        )
        text = result["text"].strip()
        return text
    except Exception as e:
        print(f"Lỗi trong speech-to-text: {e}")
        # Fallback nhanh
        try:
            result = whisper_model.transcribe(
                audio_path,
                language="vi",
                fp16=torch.cuda.is_available(),
                verbose=False,
                beam_size=1,
            )
            return result["text"].strip()
        except:
            return ""


## 5. Hàm xử lý với Bank Model


In [9]:
def process_with_model_stream(text: str):
    """
    Xử lý text với Bank Model với streaming (yield từng phần text)
    Sử dụng TextIteratorStreamer để stream response
    """
    from transformers import TextIteratorStreamer
    from threading import Thread
    import queue
    
    # Sử dụng global để truy cập model và processor
    global model, processor
    
    # Kiểm tra model và processor đã được load chưa
    if 'model' not in globals() or model is None:
        yield "Xin lỗi, model chưa được load. Vui lòng chạy cell load model trước."
        return
    
    if 'processor' not in globals() or processor is None:
        yield "Xin lỗi, processor chưa được load. Vui lòng chạy cell load model trước."
        return
    
    if not text.strip():
        yield "Xin lỗi, tôi không hiểu. Bạn có thể viết lại được không?"
        return
    
    try:
        # Tạo messages cho model - theo chuẩn Qwen3
        # Thêm system message để định nghĩa AI là tư vấn ngân hàng
        # Theo tài liệu: https://qwen.readthedocs.io/en/latest/getting_started/quickstart.html
        messages = [
            {
                "role": "system",
                "content": "Bạn là một chuyên gia tư vấn ngân hàng chuyên nghiệp và thân thiện. Nhiệm vụ của bạn là trả lời mọi câu hỏi liên quan đến ngân hàng, bao gồm: các sản phẩm và dịch vụ ngân hàng, tài khoản, thẻ, vay vốn, tiết kiệm, đầu tư, bảo hiểm ngân hàng, quy trình giao dịch, phí dịch vụ, và các vấn đề tài chính cá nhân. Hãy trả lời một cách rõ ràng, chính xác và hữu ích. Nếu không chắc chắn về thông tin, hãy đề xuất khách hàng liên hệ trực tiếp với ngân hàng để được tư vấn chi tiết hơn."
            },
            {
                "role": "user",
                "content": text
            }
        ]
        
        # Process và generate - theo đúng chuẩn Qwen3 từ tài liệu
        # Theo tài liệu: dùng apply_chat_template với tokenize=False, sau đó tokenize riêng
        tokenizer = processor.tokenizer
        
        # Bước 1: Apply chat template để lấy text (không tokenize)
        text_formatted = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        
        # Bước 2: Tokenize text đã format
        model_inputs = tokenizer([text_formatted], return_tensors="pt")
        
        # Đổi tên để dùng trong code cũ
        inputs = model_inputs
        
        # Đảm bảo input_ids là tensor 2D
        if len(inputs["input_ids"].shape) != 2:
            inputs["input_ids"] = inputs["input_ids"].unsqueeze(0)
        
        # Đảm bảo model và inputs trên GPU
        # Lưu model device trước khi có thể thay đổi
        if torch.cuda.is_available():
            try:
                model_device = next(model.parameters()).device
                if model_device.type != "cuda":
                    # Sử dụng global để cập nhật model
                    model = model.to("cuda")
                    model_device = next(model.parameters()).device
            except Exception as e:
                print(f"⚠️  Lỗi khi kiểm tra model device: {e}")
                model_device = torch.device("cuda")
                try:
                    model = model.to("cuda")
                except:
                    model_device = torch.device("cpu")
            
            device = model_device
            # Move inputs lên đúng device
            inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
        else:
            device = torch.device("cpu")
            inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
        
        # Tạo streamer với timeout và error handling tốt hơn
        # Theo tài liệu Qwen: TextIteratorStreamer với cấu hình tối ưu
        # Đảm bảo tokenizer có eos_token_id
        tokenizer = processor.tokenizer
        eos_token_id = getattr(tokenizer, 'eos_token_id', None) or getattr(tokenizer, 'pad_token_id', None)
        
        streamer = TextIteratorStreamer(
            tokenizer,
            skip_prompt=True,  # Bỏ prompt trong output
            skip_special_tokens=True,  # Bỏ special tokens
            timeout=60.0,  # Timeout 60 giây
            clean_up_tokenization_spaces=True,  # Clean up spaces (tốt cho tiếng Việt)
        )
        
        # Generate trong thread riêng
        # Theo tài liệu Qwen: cấu hình generation chuẩn và tối ưu
        model.eval()
        
        # Clear cache trước khi generate
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        # Generation config theo khuyến nghị Qwen3-Instruct (non-thinking mode)
        # Theo tài liệu: temperature=0.7, top_p=0.8, top_k=20, min_p=0
        generation_kwargs = {
            **inputs,
            "max_new_tokens": 16384,  # Theo khuyến nghị Qwen3-Instruct
            "temperature": 0.7,  # Khuyến nghị cho Qwen3-Instruct
            "top_p": 0.8,  # Khuyến nghị cho Qwen3-Instruct
            "top_k": 20,  # Khuyến nghị cho Qwen3-Instruct
            "do_sample": True,  # Bật sampling
            "pad_token_id": eos_token_id,  # Pad token
            "eos_token_id": eos_token_id,  # EOS token (quan trọng)
            "use_cache": True,  # KV cache - quan trọng cho tốc độ và memory
            "repetition_penalty": 1.1,  # Tránh lặp lại
            "streamer": streamer,  # Streamer cho streaming output
        }
        
        # Generate trong thread với error handling
        generation_error = [None]
        
        def generate_with_error_handling():
            try:
                with torch.inference_mode():  # Tối ưu inference
                    model.generate(**generation_kwargs)
            except Exception as e:
                generation_error[0] = e
                print(f"❌ Lỗi trong generation thread: {e}")
        
        thread = Thread(target=generate_with_error_handling)
        thread.daemon = True  # Daemon thread để không block khi exit
        thread.start()
        
        # Stream output với timeout handling
        generated_text = ""
        try:
            for new_text in streamer:
                if generation_error[0]:
                    raise generation_error[0]
                generated_text += new_text
                yield generated_text
        except Exception as e:
            print(f"⚠️  Lỗi trong streaming: {e}")
            if generated_text:
                yield generated_text  # Yield phần đã generate được
            else:
                yield f"Xin lỗi, có lỗi xảy ra khi generate: {str(e)}"
        finally:
            # Đảm bảo thread kết thúc
            thread.join(timeout=5.0)
            if thread.is_alive():
                print("⚠️  Generation thread vẫn chạy sau timeout")
    
    except Exception as e:
        print(f"Lỗi trong model processing: {e}")
        import traceback
        traceback.print_exc()
        yield f"Xin lỗi, có lỗi xảy ra: {str(e)}"

def process_with_model(text: str) -> str:
    """
    Xử lý text với Bank Model (tối ưu tốc độ) - chỉ text, không có ảnh
    """
    # Kiểm tra model và processor đã được load chưa
    if 'model' not in globals() or model is None:
        return "Xin lỗi, model chưa được load. Vui lòng chạy cell load model trước."
    
    if 'processor' not in globals() or processor is None:
        return "Xin lỗi, processor chưa được load. Vui lòng chạy cell load model trước."
    
    if not text.strip():
        return "Xin lỗi, tôi không nghe rõ. Bạn có thể nói lại được không?"
    
    try:
        # Tạo messages cho model - theo chuẩn Qwen3
        # Thêm system message để định nghĩa AI là tư vấn ngân hàng
        # Theo tài liệu: https://qwen.readthedocs.io/en/latest/getting_started/quickstart.html
        messages = [
            {
                "role": "system",
                "content": "Bạn là một chuyên gia tư vấn ngân hàng chuyên nghiệp và thân thiện. Nhiệm vụ của bạn là trả lời mọi câu hỏi liên quan đến ngân hàng, bao gồm: các sản phẩm và dịch vụ ngân hàng, tài khoản, thẻ, vay vốn, tiết kiệm, đầu tư, bảo hiểm ngân hàng, quy trình giao dịch, phí dịch vụ, và các vấn đề tài chính cá nhân. Hãy trả lời một cách rõ ràng, chính xác và hữu ích. Nếu không chắc chắn về thông tin, hãy đề xuất khách hàng liên hệ trực tiếp với ngân hàng để được tư vấn chi tiết hơn."
            },
            {
                "role": "user",
                "content": text
            }
        ]
        
        # Process và generate - theo đúng chuẩn Qwen3 từ tài liệu
        # Theo tài liệu: dùng apply_chat_template với tokenize=False, sau đó tokenize riêng
        tokenizer = processor.tokenizer
        
        # Bước 1: Apply chat template để lấy text (không tokenize)
        text_formatted = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        
        # Bước 2: Tokenize text đã format
        model_inputs = tokenizer([text_formatted], return_tensors="pt")
        
        # Đổi tên để dùng trong code cũ
        inputs = model_inputs
        
        # Đảm bảo input_ids là tensor 2D
        if len(inputs["input_ids"].shape) != 2:
            inputs["input_ids"] = inputs["input_ids"].unsqueeze(0)
        
        # Lấy eos_token_id
        tokenizer = processor.tokenizer
        eos_token_id = getattr(tokenizer, 'eos_token_id', None) or getattr(tokenizer, 'pad_token_id', None)
        if eos_token_id is None:
            raise ValueError("Tokenizer phải có eos_token_id hoặc pad_token_id")
        
        # Đảm bảo model và inputs trên GPU
        if torch.cuda.is_available():
            # Đảm bảo model trên GPU
            try:
                model_device = next(model.parameters()).device
                if model_device.type != "cuda":
                    print("⚠️  Model không trên GPU, đang chuyển lên GPU...")
                    model = model.to("cuda")
                    model_device = next(model.parameters()).device
                    print(f"✅ Model đã chuyển lên GPU: {torch.cuda.get_device_name(model_device.index)}")
            except Exception as e:
                print(f"⚠️  Lỗi khi kiểm tra model device: {e}")
                # Thử chuyển model lên GPU
                try:
                    model = model.to("cuda")
                    model_device = torch.device("cuda")
                    print("✅ Model đã được chuyển lên GPU")
                except:
                    model_device = torch.device("cpu")
                    print("⚠️  Không thể chuyển model lên GPU, sử dụng CPU")
            
            # Đảm bảo inputs trên GPU
            device = model_device
            inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
        else:
            device = torch.device("cpu")
            inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
            print("⚠️  Không có GPU, model đang chạy trên CPU")
        
        # Tối ưu: Sử dụng torch.inference_mode thay vì no_grad (nhanh hơn)
        # Và đảm bảo model ở eval mode
        model.eval()
        with torch.inference_mode():  # Nhanh hơn torch.no_grad()
            # Clear cache trước khi generate để tránh memory leak
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            # Generation config theo chuẩn Qwen - tối ưu hơn
            # Theo tài liệu: https://qwen.readthedocs.io/en/latest/
            # Best practices từ Qwen docs:
            # Generation config theo khuyến nghị Qwen3-Instruct (non-thinking mode)
            # Theo tài liệu: temperature=0.7, top_p=0.8, top_k=20, min_p=0
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=16384,  # Theo khuyến nghị Qwen3-Instruct
                temperature=0.7,  # Khuyến nghị cho Qwen3-Instruct
                top_p=0.8,  # Khuyến nghị cho Qwen3-Instruct
                top_k=20,  # Khuyến nghị cho Qwen3-Instruct
                do_sample=True,  # Bật sampling
                pad_token_id=eos_token_id,  # Pad token
                eos_token_id=eos_token_id,  # EOS token (quan trọng)
                use_cache=True,  # KV cache - quan trọng cho tốc độ và memory
                repetition_penalty=1.1,  # Tránh lặp lại
                output_scores=False,  # Không cần scores để tăng tốc
                return_dict_in_generate=False,  # Không cần dict để tăng tốc
            )
        
        # Decode response - theo đúng chuẩn Qwen3 từ tài liệu
        # Theo tài liệu: https://qwen.readthedocs.io/en/latest/getting_started/quickstart.html
        # Lấy output_ids từ vị trí sau input_length
        output_ids = generated_ids[0][len(inputs.input_ids[0]):].tolist()
        
        # Decode với cấu hình tối ưu
        output_text = tokenizer.decode(output_ids, skip_special_tokens=True)
        
        # Post-process: Loại bỏ các ký tự thừa và normalize
        output_text = output_text.strip()
        
        # Loại bỏ các ký tự control và whitespace thừa
        import re
        output_text = re.sub(r'\s+', ' ', output_text)  # Normalize whitespace
        output_text = output_text.strip()
        
        # Kiểm tra và xử lý empty response
        if not output_text:
            return "Xin lỗi, tôi không thể tạo response. Vui lòng thử lại."
        
        return output_text
    
    except Exception as e:
        print(f"Lỗi trong model processing: {e}")
        return f"Xin lỗi, có lỗi xảy ra: {str(e)}"


## 6. Hàm Text-to-Speech


In [10]:
def text_to_speech(text: str, lang: str = "vi") -> str:
    """
    Chuyển đổi text thành file audio sử dụng TTS (offline, tối ưu tốc độ)
    Tối ưu: Tối ưu I/O, giảm overhead
    Có nhiều fallback: Coqui TTS -> pyttsx3 -> gTTS
    """
    if not text.strip():
        return None
    
    # Kiểm tra tts đã được load chưa
    tts_available = 'tts' in globals() and tts is not None
    tts_type_available = 'tts_type' in globals() and tts_type is not None
    
    # Nếu TTS cũ chưa được load, thử load hoặc dùng fallback
    if not tts_available:
        print("⚠️  TTS cũ chưa được load, thử các TTS khác...")
        # Thử gTTS trước (nhanh, miễn phí, cần internet)
        try:
            from gtts import gTTS
            import tempfile
            import os
            
            fd, audio_path = tempfile.mkstemp(suffix=".mp3", prefix="gtts_")
            os.close(fd)
            
            tts_gtts = gTTS(text=text, lang='vi', slow=False)
            tts_gtts.save(audio_path)
            
            if os.path.exists(audio_path) and os.path.getsize(audio_path) > 0:
                print("✅ Đã dùng gTTS (cần internet)")
                return audio_path
        except Exception as e:
            print(f"⚠️  gTTS không khả dụng: {e}")
        
        # Nếu gTTS cũng không được, trả về None
        print("❌ Không có TTS nào khả dụng!")
        return None
    
    try:
        # Giới hạn độ dài để tránh TTS quá chậm (có thể điều chỉnh)
        max_chars = 500  # Giới hạn để TTS không quá chậm
        if len(text) > max_chars:
            text = text[:max_chars] + "..."
        
        # Tối ưu: Sử dụng tempfile với mode binary để nhanh hơn
        import tempfile
        import os
        fd, audio_path = tempfile.mkstemp(suffix=".wav", prefix="tts_")
        os.close(fd)  # Đóng file descriptor ngay, chỉ cần path
        
        try:
            # Sử dụng TTS đã được load
            current_tts_type = tts_type if tts_type_available else None
            
            if current_tts_type == "pyttsx3":
                # Sử dụng pyttsx3 (nhanh nhất)
                tts.save_to_file(text, audio_path)
                tts.runAndWait()
            elif current_tts_type and current_tts_type.startswith("coqui"):
                # Sử dụng Coqui TTS với settings tối ưu tốc độ
                if current_tts_type == "coqui_xtts":
                    # XTTS v2 - hỗ trợ multilingual (tối ưu tốc độ)
                    tts.tts_to_file(
                        text=text,
                        file_path=audio_path,
                        language=lang,
                        speaker_wav=None,
                        speed=1.3,  # Tăng tốc độ phát để nhanh hơn
                    )
                else:
                    # Model khác
                    tts.tts_to_file(text=text, file_path=audio_path)
            else:
                # Fallback: thử dùng API của TTS
                if hasattr(tts, 'tts_to_file'):
                    tts.tts_to_file(text=text, file_path=audio_path)
                elif hasattr(tts, 'save_to_file'):
                    tts.save_to_file(text, audio_path)
                    if hasattr(tts, 'runAndWait'):
                        tts.runAndWait()
                else:
                    print("⚠️  TTS không hỗ trợ các method phổ biến, thử gTTS...")
                    # Fallback về gTTS
                    try:
                        from gtts import gTTS
                        tts_gtts = gTTS(text=text, lang='vi', slow=False)
                        tts_gtts.save(audio_path)
                        if os.path.exists(audio_path) and os.path.getsize(audio_path) > 0:
                            print("✅ Đã dùng gTTS (cần internet)")
                            return audio_path
                    except:
                        pass
                    return None
            
            return audio_path
        except Exception as e:
            # Cleanup nếu lỗi
            if os.path.exists(audio_path):
                try:
                    os.remove(audio_path)
                except:
                    pass
            raise e
        
    except Exception as e:
        print(f"Lỗi trong text-to-speech: {e}")
        return None


## 7. Hàm xử lý Voice Chat chính


In [ ]:
def voice_chat(audio_input, chat_history=None):
    """
    Hàm chính xử lý voice chat (tối ưu full công suất) - chỉ voice, không có ảnh
    """
    import time
    start_time = time.time()
    
    if chat_history is None:
        chat_history = []
    
    # 1. Speech-to-Text
    if audio_input is None:
        return chat_history, None, format_resources_info()
    
    user_text = speech_to_text(audio_input)
    stt_time = time.time() - start_time
    
    if not user_text:
        response_text = "Xin lỗi, tôi không nghe rõ. Bạn có thể nói lại được không?"
        chat_history.append(("[Không nghe rõ]", response_text))
        audio_output = text_to_speech(response_text)
        return chat_history, audio_output, format_resources_info()
    
    # 2. Process với model (full công suất - có thể mất vài giây nhưng chất lượng cao)
    model_start = time.time()
    response_text = process_with_model(user_text)
    model_time = time.time() - model_start
    
    # 3. Cập nhật chat history ngay lập tức
    chat_history.append((user_text, response_text))
    
    # 4. Text-to-Speech
    tts_start = time.time()
    audio_output = text_to_speech(response_text)
    tts_time = time.time() - tts_start
    
    total_time = time.time() - start_time
    
    # 5. Cập nhật thông tin tài nguyên
    resources_info = format_resources_info()
    
    # Debug: In thời gian xử lý và thông tin
    print(f"⚡ Tốc độ: STT={stt_time:.2f}s, Model={model_time:.2f}s, TTS={tts_time:.2f}s, Total={total_time:.2f}s")
    print(f"📝 Response length: {len(response_text)} ký tự")
    
    return chat_history, audio_output, resources_info


## 8. Tạo Gradio Interface


In [12]:
# Tạo Gradio interface - Text Chat + Voice Input với Streaming (bỏ TTS)
with gr.Blocks(title="Chat với Bank Model - Text & Voice Input") as demo:
    gr.Markdown("""
    # 💬 Chat với Bank Model - Text & Voice Input
    
    Chat bằng text HOẶC giọng nói với AI model, response được stream realtime!
    - ✍️ Nhập text và nhấn Enter hoặc nút Gửi
    - 🎙️ Hoặc nói vào microphone
    - ⚡ Response được stream từng phần (không cần chờ)
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                label="💬 Lịch sử chat",
                height=400,
                show_label=True,
                type="tuples",
                allow_tags=False
            )
        
        with gr.Column(scale=1):
            # Tab cho Text và Voice input
            with gr.Tabs():
                with gr.Tab("✍️ Text Input"):
                    text_input = gr.Textbox(
                        label="Nhập câu hỏi",
                        placeholder="Nhập câu hỏi của bạn ở đây...",
                        lines=3,
                        show_label=True
                    )
                    text_submit_btn = gr.Button("Gửi Text", variant="primary", size="lg")
                
                with gr.Tab("🎙️ Voice Input"):
                    audio_input = gr.Audio(
                        sources=["microphone"],
                        type="filepath",
                        label="Nói vào đây",
                        show_label=True
                    )
                    audio_submit_btn = gr.Button("Gửi Voice", variant="primary", size="lg")
            
            clear_btn = gr.Button("Xóa lịch sử", variant="secondary")
            
            # Hiển thị tài nguyên hệ thống
            resources_display = gr.Markdown(
                value=format_resources_info(),
                label="📊 Tài nguyên hệ thống"
            )
    
    # Hàm xử lý text input với streaming
    def chat_text_stream(user_text, history):
        if not user_text or not user_text.strip():
            return history, ""  # Return empty string để clear text input
        
        # Thêm user message
        if history is None:
            history = []
        history.append((user_text.strip(), None))
        
        # Stream response
        response_text = ""
        try:
            for partial_response in process_with_model_stream(user_text.strip()):
                response_text = partial_response
                history[-1] = (user_text.strip(), response_text)
                yield history, ""  # Clear text input sau mỗi update
        except Exception as e:
            print(f"Lỗi trong chat_stream: {e}")
            response_text = f"Xin lỗi, có lỗi xảy ra: {str(e)}"
            history[-1] = (user_text.strip(), response_text)
        
        # Final yield - clear text input
        yield history, ""
    
    # Hàm xử lý voice input với streaming
    def chat_voice_stream(audio_input, history):
        if audio_input is None:
            return history
        
        # Speech-to-Text
        if 'whisper_model' not in globals() or whisper_model is None:
            error_msg = "Whisper model chưa được load"
            if history is None:
                history = []
            history.append(("[Lỗi]", error_msg))
            return history
        
        user_text = speech_to_text(audio_input)
        
        if not user_text or not user_text.strip():
            error_msg = "Xin lỗi, tôi không nghe rõ. Bạn có thể nói lại được không?"
            if history is None:
                history = []
            history.append(("[Không nghe rõ]", error_msg))
            return history
        
        # Thêm user message
        if history is None:
            history = []
        history.append((user_text.strip(), None))
        
        # Stream response
        response_text = ""
        try:
            for partial_response in process_with_model_stream(user_text.strip()):
                response_text = partial_response
                history[-1] = (user_text.strip(), response_text)
                yield history
        except Exception as e:
            print(f"Lỗi trong chat_voice_stream: {e}")
            response_text = f"Xin lỗi, có lỗi xảy ra: {str(e)}"
            history[-1] = (user_text.strip(), response_text)
            yield history
    
    # Event handlers
    text_submit_btn.click(
        fn=chat_text_stream,
        inputs=[text_input, chatbot],
        outputs=[chatbot, text_input],
        show_progress=False
    )
    
    text_input.submit(
        fn=chat_text_stream,
        inputs=[text_input, chatbot],
        outputs=[chatbot, text_input],
        show_progress=False
    )
    
    audio_submit_btn.click(
        fn=chat_voice_stream,
        inputs=[audio_input, chatbot],
        outputs=[chatbot],
        show_progress=False
    )
    
    audio_input.stop_recording(
        fn=chat_voice_stream,
        inputs=[audio_input, chatbot],
        outputs=[chatbot],
        show_progress=False
    )
    
    clear_btn.click(
        fn=lambda: [],
        outputs=[chatbot]
    )
    
    # Auto-refresh tài nguyên khi load page
    demo.load(
        fn=lambda: format_resources_info(),
        inputs=None,
        outputs=resources_display
    )
    
    gr.Markdown("""
    ### Hướng dẫn sử dụng:
    1. **Text Input**: 
       - ✍️ Nhập câu hỏi vào ô text
       - ⏎ Nhấn Enter hoặc nút "Gửi Text"
    
    2. **Voice Input**:
       - 🎙️ Nhấn nút microphone và bắt đầu nói
       - ⏹️ Dừng recording hoặc nhấn "Gửi Voice"
    
    3. ⚡ Xem response được stream từng phần (không cần chờ)
    4. 📊 Xem tài nguyên hệ thống ở bên phải
    
    ### ⚡ Tính năng:
    - Response được stream realtime, không cần chờ toàn bộ
    - Hỗ trợ cả text và voice input
    - Model chạy trên GPU để tối ưu tốc độ
    - Bỏ TTS (AI không trả lời bằng giọng nói)
    
    ### Lưu ý:
    - Có thể dùng text HOẶC voice để input
    - Response chỉ hiển thị text (không có audio)
    - Response được stream từng token
    """)

print("✅ Gradio interface đã được tạo với Text & Voice Input + Streaming (bỏ TTS)!")


## 8. Realtime Voice Chat 1-1 (Nâng cao)

Tính năng realtime voice chat cho phép bạn nói chuyện trực tiếp với AI như một cuộc gọi điện thoại.

### Models được đề xuất:

**STT (Speech-to-Text) - Nhận dạng giọng nói:**
1. **faster-whisper** ⭐ (Khuyến nghị)
   - Nhanh hơn Whisper 4x
   - Hỗ trợ streaming realtime
   - Offline, không cần API key
   - Chính xác cao

2. **Whisper** (Hiện tại)
   - Chính xác cao nhưng chậm hơn
   - Có thể dùng với streaming mode

**TTS (Text-to-Speech) - Tổng hợp giọng nói:**
1. **Edge TTS** ⭐ (Khuyến nghị cho realtime)
   - Miễn phí, không cần API key
   - Realtime, độ trễ thấp
   - Hỗ trợ tiếng Việt tốt
   - Chất lượng cao

2. **Piper TTS**
   - Nhẹ, nhanh, offline
   - Hỗ trợ tiếng Việt
   - Phù hợp cho realtime

3. **Coqui XTTS v2** (Đã có)
   - Chất lượng cao nhưng chậm hơn
   - Phù hợp cho non-realtime


In [13]:
# Cài đặt các thư viện cho Realtime Voice Chat
print("🔧 Đang cài đặt các thư viện cho Realtime Voice Chat...")

import subprocess
import sys

# faster-whisper - STT nhanh hơn, hỗ trợ streaming
print("\n1. Cài đặt faster-whisper (STT nhanh hơn Whisper 4x)...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "faster-whisper"], check=False)

# Edge TTS - TTS realtime miễn phí từ Microsoft
print("\n2. Cài đặt Edge TTS (TTS realtime miễn phí)...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "edge-tts"], check=False)

# nest_asyncio - Để tránh lỗi "event loop is already running"
print("\n3. Cài đặt nest-asyncio (để xử lý async trong notebook)...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "nest-asyncio"], check=False)

# gTTS - Google Text-to-Speech (fallback, cần internet)
print("\n4. Cài đặt gTTS (Google TTS - fallback, cần internet)...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "gtts"], check=False)

# Piper TTS - TTS nhẹ, nhanh, offline (tùy chọn)
print("\n5. Cài đặt Piper TTS (TTS nhẹ, offline - tùy chọn)...")
try:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "piper-tts"], check=False)
    print("   ✅ Piper TTS đã được cài đặt")
except:
    print("   ⚠️  Không thể cài Piper TTS (không bắt buộc)")

# Kiểm tra các thư viện đã cài
print("\n✅ Đang kiểm tra các thư viện...")
try:
    import faster_whisper
    print("   ✅ faster-whisper: OK")
except ImportError:
    print("   ❌ faster-whisper: Chưa cài")

try:
    import edge_tts
    print("   ✅ edge-tts: OK")
except ImportError:
    print("   ❌ edge-tts: Chưa cài")

try:
    import nest_asyncio
    print("   ✅ nest-asyncio: OK")
except ImportError:
    print("   ❌ nest-asyncio: Chưa cài")

try:
    from gtts import gTTS
    print("   ✅ gtts: OK")
except ImportError:
    print("   ⚠️  gtts: Chưa cài (không bắt buộc)")

try:
    import piper
    print("   ✅ piper-tts: OK")
except ImportError:
    print("   ⚠️  piper-tts: Chưa cài (không bắt buộc)")

print("\n✅ Hoàn tất cài đặt!")


In [14]:
# Load và khởi tạo các model cho Realtime Voice Chat

# 1. faster-whisper cho STT realtime
print("🎙️ Đang load faster-whisper model (STT realtime)...")
faster_whisper_model = None

try:
    from faster_whisper import WhisperModel
    
    # Chọn model size: tiny, base, small, medium, large
    # tiny: nhanh nhất (~39M), base: cân bằng (~74M), small: chính xác hơn (~244M)
    # Cho realtime, khuyến nghị: tiny hoặc base
    whisper_model_size = "base"  # Có thể đổi thành "tiny" để nhanh hơn hoặc "small" để chính xác hơn
    
    if torch.cuda.is_available():
        try:
            faster_whisper_model = WhisperModel(
                whisper_model_size,
                device="cuda",
                compute_type="float16"  # Nhanh hơn trên GPU
            )
            print(f"✅ faster-whisper đã load trên GPU (model: {whisper_model_size})")
            print(f"   GPU: {torch.cuda.get_device_name(0)}")
        except Exception as e:
            print(f"⚠️  Không thể load trên GPU: {e}, thử CPU...")
            faster_whisper_model = WhisperModel(
                whisper_model_size,
                device="cpu",
                compute_type="int8"  # Tối ưu cho CPU
            )
            print(f"✅ faster-whisper đã load trên CPU (model: {whisper_model_size})")
    else:
        faster_whisper_model = WhisperModel(
            whisper_model_size,
            device="cpu",
            compute_type="int8"  # Tối ưu cho CPU
        )
        print(f"✅ faster-whisper đã load trên CPU (model: {whisper_model_size})")
    
    # Test model với một đoạn audio ngắn (nếu có)
    print("   ℹ️  Model đã sẵn sàng cho realtime STT")
    
except ImportError:
    print("⚠️  faster-whisper chưa được cài, sẽ dùng Whisper thông thường")
    print("   💡 Chạy cell cài đặt dependencies trước")
    faster_whisper_model = None
except Exception as e:
    print(f"⚠️  Lỗi khi load faster-whisper: {e}")
    print("   💡 Sẽ dùng Whisper thông thường làm fallback")
    faster_whisper_model = None

# 2. Edge TTS cho TTS realtime
print("\n🔊 Đang khởi tạo Edge TTS (TTS realtime)...")
edge_tts_available = False
edge_tts_voice = None

try:
    import edge_tts
    import asyncio
    
    # Lấy danh sách giọng nói tiếng Việt
    async def get_voices():
        try:
            voices = await edge_tts.list_voices()
            vietnamese_voices = [v for v in voices if "vi" in v["Locale"].lower()]
            return vietnamese_voices
        except Exception as e:
            print(f"   ⚠️  Lỗi khi lấy danh sách giọng nói: {e}")
            return []
    
    # Chọn giọng nói tiếng Việt
    try:
        # Tạo event loop mới nếu chưa có
        try:
            loop = asyncio.get_event_loop()
        except RuntimeError:
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
        
        voices = loop.run_until_complete(get_voices())
        
        if voices:
            # Ưu tiên chọn giọng nữ (thường có tên chứa "Female" hoặc "HoaiMy")
            vietnamese_voice = None
            preferred_voices = ["HoaiMy", "female", "nữ"]
            
            for preferred in preferred_voices:
                for v in voices:
                    if preferred.lower() in v["ShortName"].lower():
                        vietnamese_voice = v["ShortName"]
                        print(f"   ✅ Tìm thấy giọng ưu tiên: {vietnamese_voice}")
                        break
                if vietnamese_voice:
                    break
            
            if not vietnamese_voice:
                vietnamese_voice = voices[0]["ShortName"]
                print(f"   ✅ Sử dụng giọng đầu tiên: {vietnamese_voice}")
            
            edge_tts_voice = vietnamese_voice
            edge_tts_available = True
            print(f"✅ Edge TTS đã sẵn sàng (giọng: {edge_tts_voice})")
        else:
            # Fallback: dùng giọng mặc định
            edge_tts_voice = "vi-VN-HoaiMyNeural"  # Giọng nữ tiếng Việt
            edge_tts_available = True
            print(f"✅ Edge TTS đã sẵn sàng (giọng mặc định: {edge_tts_voice})")
            print("   ⚠️  Không tìm thấy danh sách giọng, dùng giọng mặc định")
    except Exception as e:
        print(f"⚠️  Lỗi khi khởi tạo Edge TTS: {e}")
        # Fallback: dùng giọng mặc định
        edge_tts_voice = "vi-VN-HoaiMyNeural"
        edge_tts_available = True
        print(f"   ✅ Sử dụng giọng mặc định: {edge_tts_voice}")
        
except ImportError:
    print("⚠️  edge-tts chưa được cài")
    print("   💡 Chạy cell cài đặt dependencies trước")
    edge_tts_available = False
    edge_tts_voice = None
except Exception as e:
    print(f"⚠️  Lỗi khi import edge-tts: {e}")
    edge_tts_available = False
    edge_tts_voice = None

# Tóm tắt trạng thái
print("\n" + "="*60)
print("📊 Tóm tắt trạng thái models:")
print("="*60)
print(f"   STT (faster-whisper): {'✅ Sẵn sàng' if faster_whisper_model else '❌ Chưa sẵn sàng'}")
print(f"   TTS (Edge TTS): {'✅ Sẵn sàng' if edge_tts_available else '❌ Chưa sẵn sàng'}")
if edge_tts_available and edge_tts_voice:
    print(f"   Giọng TTS: {edge_tts_voice}")
print("="*60)

if not faster_whisper_model:
    print("\n⚠️  Lưu ý: faster-whisper chưa sẵn sàng, hệ thống sẽ dùng Whisper thông thường (chậm hơn)")
if not edge_tts_available:
    print("\n⚠️  Lưu ý: Edge TTS chưa sẵn sàng, hệ thống sẽ dùng TTS cũ (chậm hơn)")

print("\n✅ Hoàn tất khởi tạo models cho Realtime Voice Chat!")


In [15]:
# Hàm xử lý Realtime STT và TTS

def realtime_speech_to_text(audio_path: str) -> str:
    """
    Chuyển đổi audio thành text sử dụng faster-whisper (nhanh hơn)
    """
    if faster_whisper_model is not None:
        try:
            # faster-whisper nhanh hơn và hỗ trợ streaming
            segments, info = faster_whisper_model.transcribe(
                audio_path,
                language="vi",
                beam_size=1,  # Greedy decoding - nhanh nhất
                vad_filter=True,  # Voice Activity Detection - loại bỏ khoảng lặng
                vad_parameters=dict(min_silence_duration_ms=500)
            )
            
            # Lấy text từ segments
            text = " ".join([segment.text for segment in segments])
            return text.strip()
        except Exception as e:
            print(f"⚠️  Lỗi faster-whisper: {e}, dùng Whisper thông thường")
            # Fallback về Whisper thông thường
            return speech_to_text(audio_path)
    else:
        # Fallback về Whisper thông thường
        return speech_to_text(audio_path)

async def realtime_text_to_speech_async(text: str, voice: str = None) -> str:
    """
    Chuyển đổi text thành audio sử dụng Edge TTS (realtime, miễn phí)
    """
    if not edge_tts_available:
        # Fallback về TTS cũ
        return text_to_speech(text)
    
    try:
        import edge_tts
        import tempfile
        import os
        
        # Chọn giọng nói
        if voice is None:
            voice = edge_tts_voice
        
        # Kiểm tra voice có hợp lệ không
        if not voice:
            raise ValueError("Voice không được set")
        
        # Tạo file audio tạm
        fd, audio_path = tempfile.mkstemp(suffix=".mp3", prefix="edge_tts_")
        os.close(fd)
        
        # Generate audio với Edge TTS - sử dụng cách gọi đúng
        # Edge TTS cần text không rỗng và voice hợp lệ
        if not text or not text.strip():
            raise ValueError("Text không được rỗng")
        
        # Tạo communicate object với text đã clean
        text_clean = text.strip()
        
        # Kiểm tra voice có trong danh sách không (optional)
        communicate = edge_tts.Communicate(text_clean, voice)
        
        # Thử cách 1: Dùng save() (đơn giản nhất)
        try:
            await communicate.save(audio_path)
            # Kiểm tra file đã được tạo chưa
            if os.path.exists(audio_path) and os.path.getsize(audio_path) > 0:
                return audio_path
            else:
                raise ValueError("File audio rỗng sau khi save")
        except Exception as e1:
            print(f"   ⚠️  Lỗi khi dùng save(): {e1}")
            # Thử cách 2: Dùng stream() và ghi từng chunk
            try:
                # Xóa file cũ nếu có
                if os.path.exists(audio_path):
                    os.remove(audio_path)
                
                # Tạo lại communicate object
                communicate = edge_tts.Communicate(text_clean, voice)
                
                # Ghi từng chunk
                audio_data = b""
                async for chunk in communicate.stream():
                    if chunk["type"] == "audio":
                        audio_data += chunk["data"]
                
                # Ghi toàn bộ data vào file
                if len(audio_data) > 0:
                    with open(audio_path, "wb") as f:
                        f.write(audio_data)
                    
                    # Kiểm tra file
                    if os.path.exists(audio_path) and os.path.getsize(audio_path) > 0:
                        return audio_path
                    else:
                        raise ValueError("File audio rỗng sau khi stream")
                else:
                    raise ValueError("Không nhận được audio data")
            except Exception as e2:
                print(f"   ⚠️  Lỗi khi dùng stream(): {e2}")
                # Thử lại với voice mặc định nếu voice hiện tại lỗi
                if voice != "vi-VN-HoaiMyNeural":
                    print(f"   💡 Thử lại với voice mặc định...")
                    try:
                        communicate_default = edge_tts.Communicate(text_clean, "vi-VN-HoaiMyNeural")
                        await communicate_default.save(audio_path)
                        if os.path.exists(audio_path) and os.path.getsize(audio_path) > 0:
                            return audio_path
                    except:
                        pass
                raise e2
            
    except Exception as e:
        print(f"⚠️  Lỗi Edge TTS: {e}, thử TTS khác...")
        # Fallback về TTS cũ hoặc gTTS
        return text_to_speech(text)

def realtime_text_to_speech(text: str, voice: str = None) -> str:
    """
    Wrapper đồng bộ cho realtime_text_to_speech_async với nhiều fallback
    """
    import asyncio
    import tempfile
    import os
    
    # Thử Edge TTS trước
    if edge_tts_available:
        try:
            # Sử dụng nest_asyncio để tránh lỗi "event loop is already running"
            try:
                import nest_asyncio
                nest_asyncio.apply()
            except ImportError:
                pass
            
            try:
                loop = asyncio.get_event_loop()
                if loop.is_running():
                    # Nếu loop đang chạy, dùng run_coroutine_threadsafe
                    import concurrent.futures
                    with concurrent.futures.ThreadPoolExecutor() as executor:
                        future = executor.submit(
                            lambda: asyncio.run(realtime_text_to_speech_async(text, voice))
                        )
                        return future.result(timeout=30)
                else:
                    return loop.run_until_complete(realtime_text_to_speech_async(text, voice))
            except RuntimeError:
                # Tạo loop mới
                loop = asyncio.new_event_loop()
                asyncio.set_event_loop(loop)
                try:
                    return loop.run_until_complete(realtime_text_to_speech_async(text, voice))
                finally:
                    loop.close()
        except Exception as e:
            print(f"⚠️  Lỗi khi gọi Edge TTS: {e}, thử TTS khác...")
    
    # Fallback 1: TTS cũ (Coqui TTS hoặc pyttsx3)
    try:
        result = text_to_speech(text)
        if result:
            return result
    except Exception as e:
        print(f"⚠️  Lỗi TTS cũ: {e}, thử gTTS...")
    
    # Fallback 2: gTTS (Google Text-to-Speech) - cần internet nhưng miễn phí
    try:
        from gtts import gTTS
        import tempfile
        
        fd, audio_path = tempfile.mkstemp(suffix=".mp3", prefix="gtts_")
        os.close(fd)
        
        # Tạo audio với gTTS
        tts = gTTS(text=text, lang='vi', slow=False)
        tts.save(audio_path)
        
        if os.path.exists(audio_path) and os.path.getsize(audio_path) > 0:
            print("✅ Đã dùng gTTS (cần internet)")
            return audio_path
    except ImportError:
        print("⚠️  gTTS chưa được cài, cài đặt...")
        import subprocess
        import sys
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "gtts"], check=False)
        try:
            from gtts import gTTS
            fd, audio_path = tempfile.mkstemp(suffix=".mp3", prefix="gtts_")
            os.close(fd)
            tts = gTTS(text=text, lang='vi', slow=False)
            tts.save(audio_path)
            if os.path.exists(audio_path) and os.path.getsize(audio_path) > 0:
                print("✅ Đã dùng gTTS (cần internet)")
                return audio_path
        except:
            pass
    except Exception as e:
        print(f"⚠️  Lỗi gTTS: {e}")
    
    # Fallback 3: Piper TTS (nếu có)
    try:
        import piper
        import tempfile
        
        fd, audio_path = tempfile.mkstemp(suffix=".wav", prefix="piper_")
        os.close(fd)
        
        # Piper TTS cần model, tạm thời bỏ qua
        print("⚠️  Piper TTS cần cấu hình model, bỏ qua")
    except:
        pass
    
    # Nếu tất cả đều fail, trả về None
    print("❌ Không có TTS nào khả dụng!")
    return None

print("✅ Các hàm Realtime STT và TTS đã được tạo!")


In [16]:
# Tạo Gradio Interface cho Realtime Streaming Voice Chat 1-1
# Với TTS streaming (nói ngay khi có text) và STT streaming (hiển thị text dần)
# Auto-send sau 1 giây khi dừng nói

import threading
import time

with gr.Blocks(
    title="Realtime Streaming Voice Chat với AI Tư vấn Ngân hàng"
) as demo_realtime:
    gr.Markdown("""
    # 🎙️ Realtime Streaming Voice Chat với AI Tư vấn Ngân hàng
    
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white;">
    <h2 style="color: white; margin: 0;">Nói chuyện trực tiếp với AI như cuộc gọi điện thoại!</h2>
    </div>
    
    ### ⚡ Tính năng:
    - 🎤 **Voice Input**: Nhấn nút để ghi âm, nhấn Stop để gửi
    - 🤖 **Response Streaming**: AI trả lời từng token (hiển thị text)
    - 🔊 **TTS**: AI phát âm khi response hoàn tất (dùng gTTS)
    """)
    
    with gr.Row():
        with gr.Column(scale=3):
            # Chatbot với giao diện đẹp
            chatbot_realtime = gr.Chatbot(
                label="💬 Cuộc trò chuyện",
                height=450,
                show_label=True,
                type="messages",  # Dùng messages format thay vì tuples
                allow_tags=False
                # Note: show_copy_button đã deprecated, bỏ qua nếu không hỗ trợ
            )
            
            with gr.Row():
                # Text hiển thị khi đang nói (STT streaming) - giao diện đẹp hơn
                stt_streaming_text = gr.Textbox(
                    label="🎤 Đang nghe bạn nói...",
                    placeholder="Text sẽ hiển thị dần ở đây khi bạn nói...",
                    lines=2,
                    show_label=True,
                    interactive=False,
                    container=True,
                    scale=3
                )
                
                # Status indicator với icon
                status_display = gr.Markdown(
                    value="<div style='padding: 10px; background: #f0f0f0; border-radius: 5px;'>💡 **Sẵn sàng** - Nhấn microphone và bắt đầu nói</div>"
                )
            
            # Audio output ẩn (tự động phát)
            audio_output_realtime = gr.Audio(
                type="filepath",
                visible=True,  # Hiển thị để user có thể thấy và control
                autoplay=True,  # Tự động phát
                label="🔊 Audio Response"
            )
        
        with gr.Column(scale=1):
            # Voice input với giao diện đẹp
            with gr.Group():
                gr.Markdown("### 🎙️ Điều khiển")
                audio_input_realtime = gr.Audio(
                    sources=["microphone"],
                    type="filepath",
                    label="Nói vào đây (nhấn để ghi âm)",
                    show_label=True
                    # Bỏ streaming=True để chỉ ghi âm khi nhấn nút
                )
                gr.Markdown("""
                <div style="font-size: 12px; color: #666; padding: 10px; background: #f9f9f9; border-radius: 5px;">
                💡 **Hướng dẫn**:<br>
                • Nhấn nút microphone để bắt đầu ghi âm<br>
                • Nói xong thì nhấn Stop để gửi<br>
                • AI sẽ trả lời và phát âm khi hoàn tất
                </div>
                """)
            
            # Buttons
            clear_chat_btn = gr.Button(
                "🗑️ Xóa lịch sử", 
                variant="secondary", 
                size="lg",
                scale=1
            )
            
            # Hiển thị tài nguyên hệ thống
            resources_display_realtime = gr.Markdown(
                value=format_resources_info(),
                label="📊 Tài nguyên hệ thống"
            )
    
    # Biến global để lưu trữ audio chunks cho TTS streaming
    audio_queue = []
    last_audio_time = 0
    
    # Hàm helper để normalize audio input từ Gradio
    def normalize_audio_input(audio_input):
        """
        Chuyển đổi audio input từ Gradio thành file path
        Gradio có thể trả về: string path, tuple (sample_rate, audio_data), hoặc dict
        """
        if audio_input is None:
            return None
        
        import os
        import tempfile
        import soundfile as sf
        import numpy as np
        
        # Nếu là string path
        if isinstance(audio_input, str):
            # Kiểm tra file có tồn tại và có thể đọc được không
            if os.path.exists(audio_input) and os.path.getsize(audio_input) > 0:
                # Đợi một chút để đảm bảo file được ghi đầy đủ
                import time
                time.sleep(0.1)
                return audio_input
            else:
                return None
        
        # Nếu là tuple (sample_rate, audio_data)
        elif isinstance(audio_input, tuple) and len(audio_input) == 2:
            sample_rate, audio_data = audio_input
            # Convert numpy array thành file WAV
            fd, temp_path = tempfile.mkstemp(suffix=".wav")
            os.close(fd)
            try:
                sf.write(temp_path, audio_data, sample_rate)
                return temp_path
            except Exception as e:
                print(f"Lỗi khi convert audio tuple: {e}")
                if os.path.exists(temp_path):
                    os.remove(temp_path)
                return None
        
        # Nếu là dict (Gradio mới)
        elif isinstance(audio_input, dict):
            if 'path' in audio_input:
                path = audio_input['path']
                if os.path.exists(path) and os.path.getsize(path) > 0:
                    import time
                    time.sleep(0.1)
                    return path
            elif 'data' in audio_input:
                # Có audio data, cần convert
                return normalize_audio_input(audio_input['data'])
            return None
        
        return None
    
    # Hàm xử lý STT streaming (hiển thị text dần khi đang nói)
    def stt_streaming_handler(audio_input):
        """
        Xử lý STT streaming - hiển thị text dần khi đang nói
        """
        if audio_input is None:
            return ""
        
        try:
            # Normalize audio input thành file path
            audio_path = normalize_audio_input(audio_input)
            if audio_path is None or not os.path.exists(audio_path):
                return ""
            
            # Kiểm tra file có dữ liệu không
            if os.path.getsize(audio_path) == 0:
                return ""
            
            # Dùng faster-whisper với streaming (nếu có)
            if faster_whisper_model is not None:
                # Transcribe với streaming - lấy segments ngay
                segments, info = faster_whisper_model.transcribe(
                    audio_path,
                    language="vi",
                    beam_size=1,
                    vad_filter=True,
                    vad_parameters=dict(min_silence_duration_ms=300)  # Nhạy hơn
                )
                
                # Lấy text từ segments - hiển thị dần
                text_parts = []
                for segment in segments:
                    text_parts.append(segment.text)
                
                text = " ".join(text_parts)
                return text.strip()
            else:
                # Fallback về Whisper thông thường
                return speech_to_text(audio_path)
        except Exception as e:
            print(f"Lỗi STT streaming: {e}")
            import traceback
            traceback.print_exc()
            return ""
    
    # Hàm TTS đơn giản chỉ dùng gTTS
    def simple_gtts(text: str) -> str:
        """
        Chuyển đổi text thành audio chỉ dùng gTTS (bỏ các TTS khác)
        """
        if not text or not text.strip():
            print("⚠️  simple_gtts: Text rỗng")
            return None
        
        try:
            from gtts import gTTS
            import tempfile
            import os
            
            print(f"🔊 Đang tạo audio với gTTS cho text: {text[:50]}...")
            
            fd, audio_path = tempfile.mkstemp(suffix=".mp3", prefix="gtts_")
            os.close(fd)
            
            # Tạo audio với gTTS
            tts = gTTS(text=text.strip(), lang='vi', slow=False)
            tts.save(audio_path)
            
            # Kiểm tra file đã được tạo chưa
            if os.path.exists(audio_path):
                file_size = os.path.getsize(audio_path)
                if file_size > 0:
                    print(f"✅ Đã tạo audio thành công: {audio_path} ({file_size} bytes)")
                    return audio_path
                else:
                    print(f"⚠️  File audio rỗng: {audio_path}")
                    return None
            else:
                print(f"⚠️  File audio không tồn tại: {audio_path}")
                return None
        except ImportError:
            print("⚠️  gTTS chưa được cài, đang cài đặt...")
            import subprocess
            import sys
            subprocess.run([sys.executable, "-m", "pip", "install", "-q", "gtts"], check=False)
            try:
                from gtts import gTTS
                import tempfile
                import os
                fd, audio_path = tempfile.mkstemp(suffix=".mp3", prefix="gtts_")
                os.close(fd)
                tts = gTTS(text=text.strip(), lang='vi', slow=False)
                tts.save(audio_path)
                if os.path.exists(audio_path) and os.path.getsize(audio_path) > 0:
                    print(f"✅ Đã tạo audio thành công sau khi cài gTTS: {audio_path}")
                    return audio_path
                else:
                    print(f"⚠️  Không thể tạo audio sau khi cài gTTS")
                    return None
            except Exception as e:
                print(f"⚠️  Lỗi khi cài/cài gTTS: {e}")
                import traceback
                traceback.print_exc()
                return None
        except Exception as e:
            print(f"⚠️  Lỗi gTTS: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    # Hàm xử lý realtime voice chat - chỉ TTS khi response hoàn tất
    def realtime_voice_chat_streaming(audio_input, history):
        """
        Xử lý realtime voice chat:
        - STT: Chuyển đổi audio thành text
        - Model: Stream response từng token (chỉ hiển thị text)
        - TTS: Chỉ phát âm khi response hoàn tất (dùng gTTS)
        """
        if audio_input is None:
            status_html = "<div style='padding: 10px; background: #e8f5e9; border-radius: 5px;'>💡 **Sẵn sàng** - Nhấn microphone và bắt đầu nói</div>"
            return history, "", None, status_html
        
        # 1. STT - Chuyển đổi audio thành text
        user_text = stt_streaming_handler(audio_input)
        
        if not user_text or not user_text.strip():
            status_html = "<div style='padding: 10px; background: #fff3e0; border-radius: 5px;'>⚠️ **Không nghe rõ** - Vui lòng nói lại</div>"
            return history, "", None, status_html
        
        # 2. Thêm user message vào history (format messages)
        if history is None:
            history = []
        history.append({"role": "user", "content": user_text.strip()})
        
        status_html = f"<div style='padding: 10px; background: #e3f2fd; border-radius: 5px;'>🤖 **AI đang suy nghĩ** về: '{user_text.strip()[:40]}...'</div>"
        yield history, "", None, status_html  # Clear STT text, update status
        
        # 3. Process với model (streaming) - chỉ hiển thị text, không TTS streaming
        response_text = ""
        
        # Thêm assistant message với content rỗng để update sau
        history.append({"role": "assistant", "content": ""})
        
        try:
            # Stream response từng token - chỉ update UI, không TTS
            for partial_response in process_with_model_stream(user_text.strip()):
                response_text = partial_response
                history[-1] = {"role": "assistant", "content": response_text}
                
                # Chỉ update UI, không TTS streaming
                status_html = f"<div style='padding: 10px; background: #e3f2fd; border-radius: 5px;'>🤖 **AI đang trả lời**... ({len(response_text)} ký tự)</div>"
                yield history, "", None, status_html
            
            # 4. Khi response hoàn tất, mới gọi TTS (gTTS)
            if response_text and response_text.strip():
                print(f"🔊 Bắt đầu tạo audio cho response: {response_text[:100]}...")
                status_html = "<div style='padding: 10px; background: #f3e5f5; border-radius: 5px;'>🔊 **Đang tạo audio**...</div>"
                yield history, "", None, status_html
                
                try:
                    audio_output = simple_gtts(response_text.strip())
                    print(f"🔊 Kết quả simple_gtts: {audio_output}")
                    if audio_output:
                        # Kiểm tra file có tồn tại không
                        import os
                        if os.path.exists(audio_output):
                            file_size = os.path.getsize(audio_output)
                            print(f"✅ Audio file tồn tại: {audio_output} ({file_size} bytes)")
                            status_html = "<div style='padding: 10px; background: #e8f5e9; border-radius: 5px;'>✅ **Hoàn tất** - Đang phát audio</div>"
                            yield history, "", audio_output, status_html
                        else:
                            print(f"⚠️  Audio file không tồn tại: {audio_output}")
                            status_html = "<div style='padding: 10px; background: #fff3e0; border-radius: 5px;'>⚠️ **File audio không tồn tại**</div>"
                            yield history, "", None, status_html
                    else:
                        print("⚠️  simple_gtts trả về None")
                        status_html = "<div style='padding: 10px; background: #fff3e0; border-radius: 5px;'>⚠️ **Không thể tạo audio** - Nhưng đã có text response</div>"
                        yield history, "", None, status_html
                except Exception as e:
                    print(f"⚠️  Lỗi TTS: {e}")
                    import traceback
                    traceback.print_exc()
                    status_html = f"<div style='padding: 10px; background: #fff3e0; border-radius: 5px;'>⚠️ **Lỗi TTS**: {str(e)}</div>"
                    yield history, "", None, status_html
            else:
                status_html = "<div style='padding: 10px; background: #e8f5e9; border-radius: 5px;'>✅ **Hoàn tất** - Sẵn sàng cho câu hỏi tiếp theo</div>"
                yield history, "", None, status_html
            
        except Exception as e:
            print(f"Lỗi trong model processing: {e}")
            response_text = f"Xin lỗi, có lỗi xảy ra: {str(e)}"
            history[-1] = {"role": "assistant", "content": response_text}
            status_html = f"<div style='padding: 10px; background: #ffebee; border-radius: 5px;'>❌ **Lỗi**: {str(e)}</div>"
            yield history, "", None, status_html
    
    # Event handler: Xử lý khi user nhấn Stop recording
    audio_input_realtime.stop_recording(
        fn=realtime_voice_chat_streaming,
        inputs=[audio_input_realtime, chatbot_realtime],
        outputs=[chatbot_realtime, stt_streaming_text, audio_output_realtime, status_display],
        show_progress=False
    )
    
    # Bỏ STT streaming - chỉ xử lý khi user nhấn Stop
    
    clear_chat_btn.click(
        fn=lambda: ([], "", None, "<div style='padding: 10px; background: #e8f5e9; border-radius: 5px;'>💡 **Sẵn sàng** - Nhấn microphone và bắt đầu nói</div>"),
        outputs=[chatbot_realtime, stt_streaming_text, audio_output_realtime, status_display]
    )
    
    # Thêm theme vào launch (thay vì Blocks constructor)
    # Sẽ được set trong cell khởi động server
    
    # Auto-refresh tài nguyên
    demo_realtime.load(
        fn=lambda: format_resources_info(),
        inputs=None,
        outputs=resources_display_realtime
    )

print("✅ Realtime Voice Chat Interface đã được tạo!")
print("💡 Sử dụng 'demo_realtime' để khởi động server realtime")



## 9. Khởi động Realtime Voice Chat Server


In [ ]:
# Khởi động Realtime Voice Chat Server

# Kiểm tra xem demo_realtime đã được tạo chưa
if 'demo_realtime' not in globals():
    print("❌ Lỗi: demo_realtime chưa được tạo. Vui lòng chạy cell tạo Realtime Voice Chat interface trước.")
    raise RuntimeError("demo_realtime chưa được tạo. Chạy cell tạo interface trước.")

# Đóng server cũ nếu đã chạy
try:
    if hasattr(demo_realtime, 'close'):
        demo_realtime.close()
        print("✅ Đã đóng server realtime cũ")
except Exception as e:
    print(f"⚠️  Không thể đóng server cũ (có thể chưa chạy): {e}")

# Khởi động server realtime mới
print("\n" + "="*60)
print("🚀 Đang khởi động Realtime Voice Chat Server...")
print("="*60)
print("📋 Thông tin server:")
print("   - Server: 0.0.0.0:7861 (port khác với server thông thường)")
print("   - Share: True (có public link)")
print("   - Debug: True")
print("   - Mode: Realtime Voice Chat 1-1")
print("="*60 + "\n")

try:
    demo_realtime.launch(
        server_name="0.0.0.0",
        server_port=7861,  # Port khác để không conflict với server thông thường
        share=True,
        debug=True,
        show_error=True
        # Note: theme không được hỗ trợ trong launch() của phiên bản Gradio này
    )
    print("\n✅ Realtime Voice Chat Server đã khởi động thành công!")
    print("💡 Bạn có thể:")
    print("   - Nói chuyện trực tiếp với AI như cuộc gọi điện thoại")
    print("   - AI sẽ nhận dạng giọng nói, trả lời và phát âm lại")
    print("   - Sử dụng local URL hoặc public URL để truy cập")
except Exception as e:
    print(f"\n❌ Lỗi khi khởi động server: {e}")
    print("💡 Thử:")
    print("   1. Kiểm tra xem port 7861 đã được sử dụng chưa")
    print("   2. Đóng server cũ nếu có")
    print("   3. Chạy lại cell này")
    raise


## 9. Khởi động Server


In [ ]:
# Khởi động server
# Trong Colab, bạn có thể dùng share=True để có public link

# Đóng server cũ nếu đã chạy (để có thể khởi động lại)
try:
    if 'demo' in globals() and hasattr(demo, 'close'):
        demo.close()
        print("Đã đóng server cũ")
except:
    pass

# Khởi động server mới
print("Đang khởi động server...")
demo.launch(
    server_name="0.0.0.0",
    server_port=7860,
    share=True,  # Tạo public link (tắt nếu không cần)
    debug=True
    # Note: theme không được hỗ trợ trong launch() của phiên bản này
)


## 10. Alternative: FastAPI Server (Nâng cao)

Nếu bạn muốn tạo một REST API server thay vì Gradio, có thể sử dụng code bên dưới.


In [ ]:
# Uncomment để sử dụng FastAPI thay vì Gradio
"""
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import FileResponse, JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import tempfile

app = FastAPI(title="Voice Chat API")

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/voice-chat")
async def voice_chat_api(
    audio: UploadFile = File(...),
    image: UploadFile = File(None)
):
    # Lưu audio tạm
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp_audio:
        tmp_audio.write(await audio.read())
        tmp_audio_path = tmp_audio.name
    
    # Lưu image tạm nếu có
    image_path = None
    if image:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as tmp_image:
            tmp_image.write(await image.read())
            image_path = tmp_image.name
    
    try:
        # Speech-to-Text
        user_text = speech_to_text(tmp_audio_path)
        
        # Process với model
        response_text = process_with_model(user_text, image_path)
        
        # Text-to-Speech
        audio_output = text_to_speech(response_text)
        
        return FileResponse(
            audio_output,
            media_type="audio/mpeg",
            headers={"X-Response-Text": response_text}
        )
    except Exception as e:
        return JSONResponse(
            status_code=500,
            content={"error": str(e)}
        )
    finally:
        # Cleanup
        if os.path.exists(tmp_audio_path):
            os.remove(tmp_audio_path)
        if image_path and os.path.exists(image_path):
            os.remove(image_path)

@app.get("/health")
async def health():
    return {"status": "ok"}

# Chạy server
# uvicorn.run(app, host="0.0.0.0", port=8000)
"""


## Lưu ý:

1. **Whisper Model**: Có thể thay đổi size từ "tiny" (nhanh, ít chính xác) đến "large" (chậm, chính xác cao)
2. **gTTS**: Miễn phí nhưng cần internet. Có thể thay bằng các TTS khác như:
   - Coqui TTS (offline, chất lượng cao)
   - Azure Cognitive Services
   - Google Cloud TTS
3. **Memory**: Model 2B khá nhẹ, nhưng nếu hết memory có thể dùng quantization
4. **Latency**: Để giảm độ trễ, có thể:
   - Dùng Whisper model nhỏ hơn
   - Cache model responses
   - Sử dụng streaming TTS


## Tối ưu hóa đã áp dụng:

### 1. **Whisper Tiny Model**
   - Thay "base" bằng "tiny" → nhanh hơn 3-4 lần, nhẹ hơn 75%
   - Tắt các tính năng không cần thiết (verbose, condition_on_previous_text)

### 2. **Model Quantization (4-bit)**
   - Giảm memory từ ~4GB xuống ~1GB
   - Tăng tốc độ inference
   - Vẫn giữ được chất lượng tốt

### 3. **Offline TTS (Coqui TTS)**
   - Không cần internet (thay vì gTTS)
   - Nhanh hơn gTTS 2-3 lần
   - Hỗ trợ nhiều ngôn ngữ
   - Có fallback models nếu cần

### 4. **Tối ưu Generation**
   - `max_new_tokens`: 512 → 256 (nhanh hơn 2x)
   - `torch.no_grad()`: Tắt gradient tracking
   - `top_p`: 0.8 → 0.9 (ít tính toán hơn)
   - `use_cache=True`: Sử dụng KV cache

### 5. **Tối ưu Whisper**
   - `fp16=True` nếu có GPU
   - Tắt verbose và các tính năng không cần

### Kết quả so sánh:
| Metric | Trước | Sau | Cải thiện |
|--------|-------|-----|-----------|
| **Whisper Speed** | ~2-3s | ~0.5-1s | **3-4x nhanh hơn** |
| **Model Memory** | ~4GB | ~1GB | **75% tiết kiệm** |
| **TTS Speed** | ~2-3s | ~0.5-1s | **2-3x nhanh hơn** |
| **Total Latency** | ~6-9s | ~2-3s | **3x nhanh hơn** |
| **Internet** | Cần (gTTS) | Không cần | **100% offline** |

### Lưu ý:
- Whisper "tiny" có thể kém chính xác hơn "base" một chút, nhưng vẫn rất tốt
- Nếu cần chính xác hơn, có thể đổi lại "base" hoặc "small"
- TTS model sẽ tự động fallback nếu model chính không load được
